<a href="https://colab.research.google.com/github/elsonyeh/computer-programming-and-application/blob/main/EX05_01_Echo_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

In [14]:
# 先安裝必要套件（只需執行一次即可）
!pip install flask pyngrok line-bot-sdk

# 匯入套件
from flask import Flask, request, abort
from pyngrok import ngrok
from google.colab import userdata

# 匯入 Line Bot SDK v3
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage
)
from linebot.v3.webhooks import MessageEvent, TextMessageContent

# 從 Secret 讀取你的密鑰
LINE_CHANNEL_ACCESS_TOKEN = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
LINE_CHANNEL_SECRET = userdata.get('LINE_CHANNEL_SECRET')
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

# 初始化 Flask App
app = Flask(__name__)

# 初始化 Line Bot SDK 設定
configuration = Configuration(access_token=LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

# 建立 webhook 路由
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)
    return 'OK'

# 設定簡單文字回覆功能
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )

# 啟動 pyngrok + Flask
if __name__ == "__main__":
    # 設定 ngrok token
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # 開 ngrok 隧道，監聽 5000 port
    public_url = ngrok.connect(5000).public_url
    print("👉 請把這個網址填到 LINE Developer Webhook URL：")
    print(f"{public_url}/callback")

    # 啟動 Flask Server
    app.run()

👉 請把這個網址填到 LINE Developer Webhook URL：
https://93d9-34-141-136-26.ngrok-free.app/callback
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
